### Prediction using AF2, only sequence with confidence score



In [241]:
def pred_classify(filepath, threshold = 0.312, output_path = "predictions_AlphaFold_disorder_with_class.txt"):
    """
    function for adding the class column into the AF2 prediction file, 
    use threshold as 0.312, greater than 0.312 -> disorder (1)
    """
    lines = []
    with open(filepath, "r") as f:
        for line in f:
            if line.startswith(">"):
                lines.append(line)
                continue
            else:
                status = int(float(re.search(r'\d*\.\d*', line).group()) > threshold)
                line = (line.strip() + "\t" + str(status) + "\n")
                lines.append(line)
            
    with open(output_path, "w") as o:
        for line in lines:
            o.write(line)

In [242]:
pred_classify(filepath = "preidctions_AlphaFold_disorder_only_confidence.txt")

In [214]:
count = 0
with open("preidctions_AlphaFold_disorder_only_confidence.txt", "r") as f:
    for line in f:
        if line.startswith(">"):
            count += 1

In [215]:
count

489

In [23]:
def pred_file2dict(filepath, threshold = 0.312):
    """
    function converts the AF2 prediction file into a dictionary data
    """
    import pandas as pd
    import numpy as np
    import re
    af_pred = {}
    count = 0
    #use 0.312, greater than 0.312 -> disorder (1)
    with open(filepath) as f:
        curr_dp = ""
        for line in f:
            if line.startswith(">DP"):
                curr_dp = line[1:].strip()
                #print(line)
                af_pred.setdefault(curr_dp, {"index": [], "res": [], "confi": [], "disorder": []})

            else:
                index = re.search(r'\d+', line).group()
                res = re.search(r'[A-Z]', line).group()
                confi = re.search(r'\d*\.\d*', line).group()
                status = int(float(confi) > threshold)
                #print(index, res, confi)
                af_pred[curr_dp]["index"].append(index)
                af_pred[curr_dp]["res"].append(res)
                af_pred[curr_dp]["confi"].append(confi)
                af_pred[curr_dp]["disorder"].append(status)

    return af_pred



In [239]:
pred = pred_file2dict(filepath = "preidctions_AlphaFold_disorder_only_confidence.txt")

In [277]:
len("DDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD")

36

In [320]:
from Bio import SeqIO
d1 = []
d3 = []
with open("") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        if record.id[8:15] == "DP00260":
            print(record.seq, "\n")
        #print(record.id)
        #newline = record.seq.replace("F", "D")
        #newline = newline.replace("T", "D")
            
        d1.append(record.seq)

        

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- 



In [363]:
index = []
with open("preidctions_AlphaFold_disorder_only_confidence.txt", "r") as f:
    for line in f:
        if line.startswith(">"):
            index.append(line[1:].strip())


In [365]:
len(index)

489

## Preprocess 2022 tsv file

tsv -> dict

In [323]:
disprot_2022 = pd.read_csv("DisProt_2022_06.csv", header=0)

In [368]:
disprot_2022.columns

Index(['acc', 'name', 'organism', 'ncbi_taxon_id', 'disprot_id', 'namespace',
       'consensus'],
      dtype='object')

In [390]:
disprot_dict = {}
count = 0
for index, row in disprot_2022.iterrows():
    protein_id = row['disprot_id']
    disorder = [1 if res == "D" else 0 for res in row['consensus']]
    disprot_dict.setdefault(protein_id, {'disorder': disorder})
    

In [412]:
af_pred.keys()

dict_keys(['DP02220', 'DP01392', 'DP01396', 'DP01397', 'DP01169', 'DP01303', 'DP01300', 'DP01306', 'DP01304', 'DP01462', 'DP01309', 'DP01308', 'DP01467', 'DP01464', 'DP01465', 'DP01268', 'DP01269', 'DP01261', 'DP01262', 'DP01484', 'DP01485', 'DP01486', 'DP01480', 'DP01482', 'DP01521', 'DP01520', 'DP01527', 'DP01526', 'DP01524', 'DP01413', 'DP01738', 'DP01739', 'DP01736', 'DP01737', 'DP01735', 'DP01643', 'DP01641', 'DP01646', 'DP01647', 'DP01644', 'DP01645', 'DP01648', 'DP01139', 'DP01118', 'DP01867', 'DP01461', 'DP01861', 'DP01944', 'DP01947', 'DP01940', 'DP01943', 'DP01942', 'DP01167', 'DP02162', 'DP02160', 'DP02164', 'DP02169', 'DP01898', 'DP01893', 'DP01890', 'DP01487', 'DP01559', 'DP01488', 'DP01489', 'DP01109', 'DP02271', 'DP02078', 'DP01729', 'DP02207', 'DP02205', 'DP01310', 'DP01311', 'DP01312', 'DP01313', 'DP01315', 'DP01316', 'DP01317', 'DP01319', 'DP02129', 'DP02294', 'DP02295', 'DP02296', 'DP02290', 'DP02292', 'DP01250', 'DP01253', 'DP01252', 'DP01255', 'DP01538', 'DP01531',

---

#### Example: DP02025, AF2 predicted disorder content: 14.9%; disprot database shown: 18.9%

In [423]:
(np.sum(af_pred['DP02025']['disorder'])) / len(af_pred['DP02025']['disorder'])

0.14886164623467601

In [427]:
af_pred['DP02025']['disorder'][90:114]

[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0]

In [426]:
disprot_dict['DP02025']['disorder'][90:114]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]

---

In [435]:
disport_dict_calibration = disprot_dict.copy()
for index, key in enumerate(disprot_dict):
    if key in af_pred.keys():
        continue
    else:
        disport_dict_calibration.pop(key)

In [437]:
count = 0
for index, key in enumerate(af_pred):
    if key in disport_dict_calibration.keys():
        count += 1
    else:
        print(key)

DP01505
DP01868
DP01477


### Process tsv file ->  format:id, sequence, res disorder classification
In order to make a test data set


In [109]:
import pandas as pd
import numpy as np
import json

In [111]:
with open('DisProt release_2022_06.json') as f:
    disprot_2022_json = json.load(f)

In [112]:
len(disprot_2022_json['data'])

2290

In [113]:
disprot_2022 = pd.read_csv("DisProt_2022_06.csv", header=0)[["acc", "consensus"]]

In [114]:
# used for test data set
disprot_2022_preprocessed = disprot_2022.copy()
disprot_2022_preprocessed.assign(sequence = '')

for index, row in disprot_2022.iterrows():
    newline = ''.join(["1" if res == "D" else "0" for res in row.consensus])
    disprot_2022_preprocessed.loc[index, 'consensus'] = newline    
    disprot_2022_preprocessed.loc[index, 'sequence'] = disprot_2022_json['data'][index]['sequence']
    #print(disprot_2022_preprocessed.loc[index, 'sequence'])

In [115]:
disprot_2022_preprocessed['acc'][0]

'P03265'

In [116]:
disprot_2022_preprocessed['sequence'][0]

'MASREEEQRETTPERGRGAARRPPTMEDVSSPSPSPPPPRAPPKKRMRRRIESEDEEDSSQDALVPRTPSPRPSTSAADLAIAPKKKKKRPSPKPERPPSPEVIVDSEEEREDVALQMVGFSNPPVLIKHGKGGKRTVRRLNEDDPVARGMRTQEEEEEPSEAESEITVMNPLSVPIVSAWEKGMEAARALMDKYHVDNDLKANFKLLPDQVEALAAVCKTWLNEEHRGLQLTFTSKKTFVTMMGRFLQAYLQSFAEVTYKHHEPTGCALWLHRCAEIEGELKCLHGSIMINKEHVIEMDVTSENGQRALKEQSSKAKIVKNRWGRNVVQISNTDARCCVHDAACPANQFSGKSCGMFFSEGAKAQVAFKQIKAFMQALYPNAQTGHGHLLMPLRCECNSKPGHAPFLGRQLPKLTPFALSNAEDLDADLISDKSVLASVHHPALIVFQCCNPVYRNSRAQGGGPNCDFKISAPDLLNALVMVRSLWSENFTELPRMVVPEFKWSTKHQYRNVSLPVAHSDARQNPFDF'

In [117]:
disprot_2022_preprocessed['consensus'][0]

'0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000011111111111111111111111111111111111111111000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001111111111100000000000000000000000000000000000000000000000000000000000000000'

In [120]:
disprot_2022_preprocessed = disprot_2022_preprocessed[['acc', 'sequence', 'consensus']]

In [121]:
disprot_2022_preprocessed.to_csv('disprot_2022_preprocessed.csv', index=False)

### Process the alphafold prediction files (Escherichia coli)
#### for train
### -> Format:id, sequence, res disorder classification

In [1]:
from Bio.PDB import MMCIF2Dict

In [2]:
def get_plddtsocre_seq_from_af2_cif(filepath):
    """
    return the db id, sequence and disorder state for each residue (use threshold as 50, disorder1; order: 0)
    """
    file_dict = MMCIF2Dict.MMCIF2Dict(filepath)
    sequence = ''.join(file_dict['_entity_poly.pdbx_seq_one_letter_code'])
    plddt = file_dict['_ma_qa_metric_local.metric_value']
    disorder_class = ''.join(['1' if float(score) < 50 else '0' for score in plddt])
    db_id = ''.join(file_dict['_ma_target_ref_db_details.db_accession'])
    
    return db_id, sequence, disorder_class

In [131]:
import tarfile
import io
import gzip
species_tars = tarfile.open("UP000000625_83333_ECOLI_v2.tar")
total = len(species_tars.getnames())
count = 0
lines = []
for tar in species_tars:
    byte_io = io.BytesIO(gzip.GzipFile(fileobj=species_tars.extractfile(tar)).read())
    file = io.StringIO(byte_io.read().decode('UTF-8'))
    try:
        db_id, seq, score = get_plddtsocre_seq_from_af2_cif(file)
        count += 1
    except:
        print(file.read())
    newline = ">" + db_id + "\n" + seq.replace("\n") + "\n" + score + "\n"
    lines.append(newline)
    print("finish: ", count, "/", total, "\n")

finish:  1 / 8726 


finish:  1 / 8726 

finish:  2 / 8726 


finish:  2 / 8726 

finish:  3 / 8726 


finish:  3 / 8726 

finish:  4 / 8726 


finish:  4 / 8726 

finish:  5 / 8726 


finish:  5 / 8726 

finish:  6 / 8726 


finish:  6 / 8726 

finish:  7 / 8726 


finish:  7 / 8726 

finish:  8 / 8726 


finish:  8 / 8726 

finish:  9 / 8726 


finish:  9 / 8726 

finish:  10 / 8726 


finish:  10 / 8726 

finish:  11 / 8726 


finish:  11 / 8726 

finish:  12 / 8726 


finish:  12 / 8726 

finish:  13 / 8726 


finish:  13 / 8726 

finish:  14 / 8726 


finish:  14 / 8726 

finish:  15 / 8726 


finish:  15 / 8726 

finish:  16 / 8726 


finish:  16 / 8726 

finish:  17 / 8726 


finish:  17 / 8726 

finish:  18 / 8726 


finish:  18 / 8726 

finish:  19 / 8726 


finish:  19 / 8726 

finish:  20 / 8726 


finish:  20 / 8726 

finish:  21 / 8726 


finish:  21 / 8726 

finish:  22 / 8726 


finish:  22 / 8726 

finish:  23 / 8726 


finish:  23 / 8726 

finish:  24 / 8726 


finish:

finish:  190 / 8726 


finish:  190 / 8726 

finish:  191 / 8726 


finish:  191 / 8726 

finish:  192 / 8726 


finish:  192 / 8726 

finish:  193 / 8726 


finish:  193 / 8726 

finish:  194 / 8726 


finish:  194 / 8726 

finish:  195 / 8726 


finish:  195 / 8726 

finish:  196 / 8726 


finish:  196 / 8726 

finish:  197 / 8726 


finish:  197 / 8726 

finish:  198 / 8726 


finish:  198 / 8726 

finish:  199 / 8726 


finish:  199 / 8726 

finish:  200 / 8726 


finish:  200 / 8726 

finish:  201 / 8726 


finish:  201 / 8726 

finish:  202 / 8726 


finish:  202 / 8726 

finish:  203 / 8726 


finish:  203 / 8726 

finish:  204 / 8726 


finish:  204 / 8726 

finish:  205 / 8726 


finish:  205 / 8726 

finish:  206 / 8726 


finish:  206 / 8726 

finish:  207 / 8726 


finish:  207 / 8726 

finish:  208 / 8726 


finish:  208 / 8726 

finish:  209 / 8726 


finish:  209 / 8726 

finish:  210 / 8726 


finish:  210 / 8726 

finish:  211 / 8726 


finish:  211 / 8726 

finish:  2


finish:  374 / 8726 

finish:  375 / 8726 


finish:  375 / 8726 

finish:  376 / 8726 


finish:  376 / 8726 

finish:  377 / 8726 


finish:  377 / 8726 

finish:  378 / 8726 


finish:  378 / 8726 

finish:  379 / 8726 


finish:  379 / 8726 

finish:  380 / 8726 


finish:  380 / 8726 

finish:  381 / 8726 


finish:  381 / 8726 

finish:  382 / 8726 


finish:  382 / 8726 

finish:  383 / 8726 


finish:  383 / 8726 

finish:  384 / 8726 


finish:  384 / 8726 

finish:  385 / 8726 


finish:  385 / 8726 

finish:  386 / 8726 


finish:  386 / 8726 

finish:  387 / 8726 


finish:  387 / 8726 

finish:  388 / 8726 


finish:  388 / 8726 

finish:  389 / 8726 


finish:  389 / 8726 

finish:  390 / 8726 


finish:  390 / 8726 

finish:  391 / 8726 


finish:  391 / 8726 

finish:  392 / 8726 


finish:  392 / 8726 

finish:  393 / 8726 


finish:  393 / 8726 

finish:  394 / 8726 


finish:  394 / 8726 

finish:  395 / 8726 


finish:  395 / 8726 

finish:  396 / 8726 


finish:  

finish:  560 / 8726 


finish:  560 / 8726 

finish:  561 / 8726 


finish:  561 / 8726 

finish:  562 / 8726 


finish:  562 / 8726 

finish:  563 / 8726 


finish:  563 / 8726 

finish:  564 / 8726 


finish:  564 / 8726 

finish:  565 / 8726 


finish:  565 / 8726 

finish:  566 / 8726 


finish:  566 / 8726 

finish:  567 / 8726 


finish:  567 / 8726 

finish:  568 / 8726 


finish:  568 / 8726 

finish:  569 / 8726 


finish:  569 / 8726 

finish:  570 / 8726 


finish:  570 / 8726 

finish:  571 / 8726 


finish:  571 / 8726 

finish:  572 / 8726 


finish:  572 / 8726 

finish:  573 / 8726 


finish:  573 / 8726 

finish:  574 / 8726 


finish:  574 / 8726 

finish:  575 / 8726 


finish:  575 / 8726 

finish:  576 / 8726 


finish:  576 / 8726 

finish:  577 / 8726 


finish:  577 / 8726 

finish:  578 / 8726 


finish:  578 / 8726 

finish:  579 / 8726 


finish:  579 / 8726 

finish:  580 / 8726 


finish:  580 / 8726 

finish:  581 / 8726 


finish:  581 / 8726 

finish:  5

finish:  744 / 8726 


finish:  744 / 8726 

finish:  745 / 8726 


finish:  745 / 8726 

finish:  746 / 8726 


finish:  746 / 8726 

finish:  747 / 8726 


finish:  747 / 8726 

finish:  748 / 8726 


finish:  748 / 8726 

finish:  749 / 8726 


finish:  749 / 8726 

finish:  750 / 8726 


finish:  750 / 8726 

finish:  751 / 8726 


finish:  751 / 8726 

finish:  752 / 8726 


finish:  752 / 8726 

finish:  753 / 8726 


finish:  753 / 8726 

finish:  754 / 8726 


finish:  754 / 8726 

finish:  755 / 8726 


finish:  755 / 8726 

finish:  756 / 8726 


finish:  756 / 8726 

finish:  757 / 8726 


finish:  757 / 8726 

finish:  758 / 8726 


finish:  758 / 8726 

finish:  759 / 8726 


finish:  759 / 8726 

finish:  760 / 8726 


finish:  760 / 8726 

finish:  761 / 8726 


finish:  761 / 8726 

finish:  762 / 8726 


finish:  762 / 8726 

finish:  763 / 8726 


finish:  763 / 8726 

finish:  764 / 8726 


finish:  764 / 8726 

finish:  765 / 8726 


finish:  765 / 8726 

finish:  7

...


finish:  3567 / 8726 

finish:  3568 / 8726 


finish:  3568 / 8726 

finish:  3569 / 8726 


finish:  3569 / 8726 

finish:  3570 / 8726 


finish:  3570 / 8726 

finish:  3571 / 8726 


finish:  3571 / 8726 

finish:  3572 / 8726 


finish:  3572 / 8726 

finish:  3573 / 8726 


finish:  3573 / 8726 

finish:  3574 / 8726 


finish:  3574 / 8726 

finish:  3575 / 8726 


finish:  3575 / 8726 

finish:  3576 / 8726 


finish:  3576 / 8726 

finish:  3577 / 8726 


finish:  3577 / 8726 

finish:  3578 / 8726 


finish:  3578 / 8726 

finish:  3579 / 8726 


finish:  3579 / 8726 

finish:  3580 / 8726 


finish:  3580 / 8726 

finish:  3581 / 8726 


finish:  3581 / 8726 

finish:  3582 / 8726 


finish:  3582 / 8726 

finish:  3583 / 8726 


finish:  3583 / 8726 

finish:  3584 / 8726 


finish:  3584 / 8726 

finish:  3585 / 8726 


finish:  3585 / 8726 

finish:  3586 / 8726 


finish:  3586 / 8726 

finish:  3587 / 8726 


finish:  3587 / 8726 

finish:  3588 / 8726 


finish:  358

finish:  3745 / 8726 


finish:  3745 / 8726 

finish:  3746 / 8726 


finish:  3746 / 8726 

finish:  3747 / 8726 


finish:  3747 / 8726 

finish:  3748 / 8726 


finish:  3748 / 8726 

finish:  3749 / 8726 


finish:  3749 / 8726 

finish:  3750 / 8726 


finish:  3750 / 8726 

finish:  3751 / 8726 


finish:  3751 / 8726 

finish:  3752 / 8726 


finish:  3752 / 8726 

finish:  3753 / 8726 


finish:  3753 / 8726 

finish:  3754 / 8726 


finish:  3754 / 8726 

finish:  3755 / 8726 


finish:  3755 / 8726 

finish:  3756 / 8726 


finish:  3756 / 8726 

finish:  3757 / 8726 


finish:  3757 / 8726 

finish:  3758 / 8726 


finish:  3758 / 8726 

finish:  3759 / 8726 


finish:  3759 / 8726 

finish:  3760 / 8726 


finish:  3760 / 8726 

finish:  3761 / 8726 


finish:  3761 / 8726 

finish:  3762 / 8726 


finish:  3762 / 8726 

finish:  3763 / 8726 


finish:  3763 / 8726 

finish:  3764 / 8726 


finish:  3764 / 8726 

finish:  3765 / 8726 


finish:  3765 / 8726 

finish:  3766

finish:  3921 / 8726 


finish:  3921 / 8726 

finish:  3922 / 8726 


finish:  3922 / 8726 

finish:  3923 / 8726 


finish:  3923 / 8726 

finish:  3924 / 8726 


finish:  3924 / 8726 

finish:  3925 / 8726 


finish:  3925 / 8726 

finish:  3926 / 8726 


finish:  3926 / 8726 

finish:  3927 / 8726 


finish:  3927 / 8726 

finish:  3928 / 8726 


finish:  3928 / 8726 

finish:  3929 / 8726 


finish:  3929 / 8726 

finish:  3930 / 8726 


finish:  3930 / 8726 

finish:  3931 / 8726 


finish:  3931 / 8726 

finish:  3932 / 8726 


finish:  3932 / 8726 

finish:  3933 / 8726 


finish:  3933 / 8726 

finish:  3934 / 8726 


finish:  3934 / 8726 

finish:  3935 / 8726 


finish:  3935 / 8726 

finish:  3936 / 8726 


finish:  3936 / 8726 

finish:  3937 / 8726 


finish:  3937 / 8726 

finish:  3938 / 8726 


finish:  3938 / 8726 

finish:  3939 / 8726 


finish:  3939 / 8726 

finish:  3940 / 8726 


finish:  3940 / 8726 

finish:  3941 / 8726 


finish:  3941 / 8726 

finish:  3942

finish:  4098 / 8726 


finish:  4098 / 8726 

finish:  4099 / 8726 


finish:  4099 / 8726 

finish:  4100 / 8726 


finish:  4100 / 8726 

finish:  4101 / 8726 


finish:  4101 / 8726 

finish:  4102 / 8726 


finish:  4102 / 8726 

finish:  4103 / 8726 


finish:  4103 / 8726 

finish:  4104 / 8726 


finish:  4104 / 8726 

finish:  4105 / 8726 


finish:  4105 / 8726 

finish:  4106 / 8726 


finish:  4106 / 8726 

finish:  4107 / 8726 


finish:  4107 / 8726 

finish:  4108 / 8726 


finish:  4108 / 8726 

finish:  4109 / 8726 


finish:  4109 / 8726 

finish:  4110 / 8726 


finish:  4110 / 8726 

finish:  4111 / 8726 


finish:  4111 / 8726 

finish:  4112 / 8726 


finish:  4112 / 8726 

finish:  4113 / 8726 


finish:  4113 / 8726 

finish:  4114 / 8726 


finish:  4114 / 8726 

finish:  4115 / 8726 


finish:  4115 / 8726 

finish:  4116 / 8726 


finish:  4116 / 8726 

finish:  4117 / 8726 


finish:  4117 / 8726 

finish:  4118 / 8726 


finish:  4118 / 8726 

finish:  4119

finish:  4275 / 8726 


finish:  4275 / 8726 

finish:  4276 / 8726 


finish:  4276 / 8726 

finish:  4277 / 8726 


finish:  4277 / 8726 

finish:  4278 / 8726 


finish:  4278 / 8726 

finish:  4279 / 8726 


finish:  4279 / 8726 

finish:  4280 / 8726 


finish:  4280 / 8726 

finish:  4281 / 8726 


finish:  4281 / 8726 

finish:  4282 / 8726 


finish:  4282 / 8726 

finish:  4283 / 8726 


finish:  4283 / 8726 

finish:  4284 / 8726 


finish:  4284 / 8726 

finish:  4285 / 8726 


finish:  4285 / 8726 

finish:  4286 / 8726 


finish:  4286 / 8726 

finish:  4287 / 8726 


finish:  4287 / 8726 

finish:  4288 / 8726 


finish:  4288 / 8726 

finish:  4289 / 8726 


finish:  4289 / 8726 

finish:  4290 / 8726 


finish:  4290 / 8726 

finish:  4291 / 8726 


finish:  4291 / 8726 

finish:  4292 / 8726 


finish:  4292 / 8726 

finish:  4293 / 8726 


finish:  4293 / 8726 

finish:  4294 / 8726 


finish:  4294 / 8726 

finish:  4295 / 8726 


finish:  4295 / 8726 

finish:  4296

In [135]:
with open("output_path.txt", "w") as o:
        for line in set(lines):
            o.write(line) 
            o.write("\n")

In [164]:
lines[0]

'>A0A385XJ53\nMASVSISCPSCSATDGVVRNGKSTAGHQRYLCSHCRKTWQLQFTYTASQPGTHQKIIDMAMNGVGCRATARIMGVGLNTI\nLRHLKNSGRSR\n1000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001\n'

### convert the output_path txt file into a csv file


In [166]:
train_data = {"acc": [], "seq":[], "disorder":[]}
index = 0
seq = ""
with open("output_path_unique_Escherichia coli.txt", 'r') as f:
    for _, line in enumerate(f):
        if line.startswith(">"):
            train_data["acc"].append(line[1:].strip())
        elif line.startswith("1") or line.startswith("0"):
            train_data["seq"].append(seq.replace("\n", ''))
            seq = ""
            train_data["disorder"].append(line.strip())
            index += 1
        else:
            
            seq += line
            

In [167]:
train_data_csv = pd.DataFrame.from_dict(train_data)

In [169]:
train_data_csv.to_csv('output_path_unique_Escherichia_coli.csv', index=False)